In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
def draw_styled_landmarks(image, results):
    # Draw face connections (use face_landmarks directly, not FACE_CONNECTIONS)
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,  # use FACEMESH_TESSELATION for face
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        ) 

    # Draw pose connections
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        ) 

    # Draw left hand connections
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        ) 

    # Draw right hand connections  
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
import os
import numpy as np

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('gym_Data')

# Actions that we try to detect
actions = np.array(['push-up', 'squart'])

# Create action directories if they don't exist
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_path):
        os.makedirs(action_path)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder =1

In [9]:
actions = np.array(['push-up', 'squart'])
DATA_PATH = os.path.join('gym_Data')
no_sequences = 30
sequence_length = 30
start_folder =1

In [11]:
for action in actions: 
    action_path = os.path.join(DATA_PATH, action)
    
    # Check if the action directory exists
    if os.path.exists(action_path):
        # Get list of existing directories
        existing_dirs = os.listdir(action_path)
        # Filter out non-integer directory names
        existing_dirs = [int(d) for d in existing_dirs if d.isdigit()]
        
        # Check if there are any existing directories
        if existing_dirs:
            dirmax = np.max(existing_dirs)
        else:
            dirmax = 0  # If no directories exist, start from 0
    else:
        dirmax = 0  # If action directory doesn't exist, start from 0
    
    # Create new directories for sequences
    for sequence in range(1, no_sequences + 1):
        try: 
            os.makedirs(os.path.join(action_path, str(dirmax + sequence)))
        except FileExistsError:
            pass  # Ignore if the directory already exists

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(300)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [19]:
cap.release()
cv2.destroyAllWindows()

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'push-up': 0, 'squart': 1}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
X = np.array(sequences)
X.shape

(60, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

(3, 2)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [20]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
2/2 [==============================] - 5s 162ms/step - loss: 0.6818 - categorical_accuracy: 0.6140
Epoch 2/500
2/2 [==============================] - 0s 146ms/step - loss: 0.6399 - categorical_accuracy: 0.5263
Epoch 3/500
2/2 [==============================] - 1s 570ms/step - loss: 0.4997 - categorical_accuracy: 0.8947
Epoch 4/500
2/2 [==============================] - 1s 706ms/step - loss: 1.6035 - categorical_accuracy: 0.8070
Epoch 5/500
2/2 [==============================] - 1s 453ms/step - loss: 2.6311 - categorical_accuracy: 0.8947
Epoch 6/500
2/2 [==============================] - 0s 110ms/step - loss: 4.2629 - categorical_accuracy: 0.5965
Epoch 7/500
2/2 [==============================] - 0s 74ms/step - loss: 1.8006 - categorical_accuracy: 0.5614
Epoch 8/500
2/2 [==============================] - 0s 158ms/step - loss: 1.1698 - categorical_accuracy: 0.8421
Epoch 9/500
2/2 [==============================] - 0s 136ms/step - loss: 3.8642 - categorical_accuracy: 0.7895
Ep

In [29]:
actions[np.argmax(y_test[2])]

'push-up'

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 1s 1s/step


array([[[2, 0],
        [0, 1]],

       [[1, 0],
        [0, 2]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

In [37]:
from scipy import stats

In [39]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [41]:
import joblib
joblib.dump(model, 'pose_model.pkl')
print("Model saved as pose_model.pkl")

INFO:tensorflow:Assets written to: ram://96097aa5-0fbc-4ad7-a1b7-a06a5a8e5060/assets
Model saved as pose_model1.pkl


In [10]:
import cv2
import numpy as np
import joblib
import mediapipe as mp

# Load the saved model
import os
model = joblib.load('pose_model.pkl')


# Initialize counts and states
pushup_count = 0
squat_count = 0
is_pushup_down = False
is_squat_down = False

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to calculate angles between joints
def calculate_angle(point1, point2, point3):
    a = np.array(point1)  # First joint
    b = np.array(point2)  # Second joint (the vertex)
    c = np.array(point3)  # Third joint

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Function to detect pose keypoints using MediaPipe
def detect_pose_keypoints(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract the necessary joints: (e.g., hip, knee, ankle, shoulder, elbow, wrist)
        pose_keypoints = {
            'hip': [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y],
            'knee': [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y],
            'ankle': [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y],
            'shoulder': [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y],
            'elbow': [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y],
            'wrist': [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
        }
        return pose_keypoints
    else:
        return None

# Function to predict pose and count repetitions
def detect_and_count_exercise(pose_keypoints, frame):
    global pushup_count, squat_count, is_pushup_down, is_squat_down
    
    # Predict the pose (e.g., 'squat', 'pushup')
    predicted_pose = model.predict([np.array(list(pose_keypoints.values())).flatten()])[0]
    
    if predicted_pose == 'squat':
        hip = pose_keypoints['hip']
        knee = pose_keypoints['knee']
        ankle = pose_keypoints['ankle']
        
        # Calculate knee angle
        knee_angle = calculate_angle(hip, knee, ankle)
        
        # Squat counting logic
        if knee_angle < 90:  # Squat down
            is_squat_down = True
        elif knee_angle > 160 and is_squat_down:  # Squat up
            squat_count += 1
            is_squat_down = False
        
        # Display squat count
        cv2.putText(frame, f'Squats: {squat_count}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    elif predicted_pose == 'pushup':
        shoulder = pose_keypoints['shoulder']
        elbow = pose_keypoints['elbow']
        wrist = pose_keypoints['wrist']
        
        # Calculate elbow angle
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        
        # Pushup counting logic
        if elbow_angle < 90:  # Pushup down
            is_pushup_down = True
        elif elbow_angle > 160 and is_pushup_down:  # Pushup up
            pushup_count += 1
            is_pushup_down = False
        
        # Display pushup count
        cv2.putText(frame, f'Pushups: {pushup_count}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the predicted pose
    cv2.putText(frame, f'Pose: {predicted_pose}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

# Function to run the video stream and count exercises
def run_exercise_counter(video_source=0):
    cap = cv2.VideoCapture(video_source)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect pose keypoints using MediaPipe
        frame = cv2.flip(frame, 1)
        pose_keypoints = detect_pose_keypoints(frame)

        # If keypoints are detected, detect pose and count exercise reps
        if pose_keypoints:
            detect_and_count_exercise(pose_keypoints, frame)

        # Display the frame
        cv2.imshow('Pose Estimation & Exercise Counter', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start the exercise counter
run_exercise_counter()



ModuleNotFoundError: No module named 'keras'

In [53]:
cap.release()
cv2.destroyAllWindows()

In [19]:
import cv2
import numpy as np
import joblib
import mediapipe as mp
from tensorflow.models import load_model

# Load the saved model with compatibility checks
def load_pose_model(model_path):
    try:
        # Try loading with joblib
        model = joblib.load(model_path)
        print("Model loaded successfully with joblib.")
    except Exception as e:
        print(f"Error loading model with joblib: {e}")
        try:
            # Try loading as a TensorFlow/Keras model
            model = load_model(model_path)
            print("Model loaded successfully with TensorFlow/Keras.")
        except Exception as inner_e:
            print(f"Error loading model with TensorFlow/Keras: {inner_e}")
            raise ValueError("Model could not be loaded. Ensure compatibility.")
    return model

# Path to your model file
model_path = 'pose_model.pkl'
model = load_pose_model(model_path)

# Initialize counts and states
pushup_count = 0
squat_count = 0
is_pushup_down = False
is_squat_down = False

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to calculate angles between joints
def calculate_angle(point1, point2, point3):
    a = np.array(point1)  # First joint
    b = np.array(point2)  # Second joint (the vertex)
    c = np.array(point3)  # Third joint

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Function to detect pose keypoints using MediaPipe
def detect_pose_keypoints(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract the necessary joints: (e.g., hip, knee, ankle, shoulder, elbow, wrist)
        pose_keypoints = {
            'hip': [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y],
            'knee': [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y],
            'ankle': [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y],
            'shoulder': [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y],
            'elbow': [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y],
            'wrist': [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
        }
        return pose_keypoints
    else:
        return None

# Function to predict pose and count repetitions
def detect_and_count_exercise(pose_keypoints, frame):
    global pushup_count, squat_count, is_pushup_down, is_squat_down
    
    # Predict the pose (e.g., 'squat', 'pushup')
    pose_data = np.array(list(pose_keypoints.values())).flatten().reshape(1, -1)
    predicted_pose = model.predict(pose_data)[0]
    
    if predicted_pose == 'squat':
        hip = pose_keypoints['hip']
        knee = pose_keypoints['knee']
        ankle = pose_keypoints['ankle']
        
        # Calculate knee angle
        knee_angle = calculate_angle(hip, knee, ankle)
        
        # Squat counting logic
        if knee_angle < 90:  # Squat down
            is_squat_down = True
        elif knee_angle > 160 and is_squat_down:  # Squat up
            squat_count += 1
            is_squat_down = False
        
        # Display squat count
        cv2.putText(frame, f'Squats: {squat_count}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    elif predicted_pose == 'pushup':
        shoulder = pose_keypoints['shoulder']
        elbow = pose_keypoints['elbow']
        wrist = pose_keypoints['wrist']
        
        # Calculate elbow angle
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        
        # Pushup counting logic
        if elbow_angle < 90:  # Pushup down
            is_pushup_down = True
        elif elbow_angle > 160 and is_pushup_down:  # Pushup up
            pushup_count += 1
            is_pushup_down = False
        
        # Display pushup count
        cv2.putText(frame, f'Pushups: {pushup_count}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the predicted pose
    cv2.putText(frame, f'Pose: {predicted_pose}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

# Function to run the video stream and count exercises
def run_exercise_counter(video_source=0):
    cap = cv2.VideoCapture(video_source)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect pose keypoints using MediaPipe
        frame = cv2.flip(frame, 1)
        pose_keypoints = detect_pose_keypoints(frame)

        # If keypoints are detected, detect pose and count exercise reps
        if pose_keypoints:
            detect_and_count_exercise(pose_keypoints, frame)

        # Display the frame
        cv2.imshow('Pose Estimation & Exercise Counter', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start the exercise counter
run_exercise_counter()


ModuleNotFoundError: No module named 'tensorflow.models'

In [13]:
!pip install opencv-python mediapipe numpy tensorflow scikit-learn joblib

  Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached ml_dtypes-0.5.1-cp310-cp310-win_amd64.whl.metadata (22 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl (375.7 MB)
Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
Using cached ml_dtypes-0.5.1-cp310-cp310-win_amd64.whl (209 kB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
Using cached tensorboard_data_server-0.7.2-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1


error: uninstall-no-record-file

Cannot uninstall tensorboard-data-server 0.6.1

The package's contents are unknown: no RECORD file was found for tensorboard-data-server.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps tensorboard-data-server==0.6.1
